### Introduction
This code analyzes the differences in performance for various CNNs (Convolutional Neural Networks) and an MLP (Multi Layer Perceptron) for the CIFAR10 dataset. The purpose is to analyze the effect of layers and hidden nodes on the efficiency and accuracy of models.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

#(MacOS user)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [ ]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

def get_random_data(x,y,sample_percent):
    total = y_train.shape[0]
    sample_amount = int(total*(sample_percent/100))
    
    # returns an array of `sample_amount` length with random values from [0,`total`] 
    random_indices = np.random.choice(total,sample_amount)
    x= x[random_indices]
    y= y[random_indices]
    return x, y


#Normalization
def normalize(x):
    return (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))


x_train, y_train = get_random_data(x_train,y_train,20)

#one hot encode
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

#normalize
x_train = normalize(x_train.astype(np.float32))
x_test = normalize(x_test.astype(np.float32))

print('train images shape:', x_train.shape)
print('train labels shape:', y_train.shape)
print('test images shape:', x_test.shape)
print('test labels shape:', y_test.shape)

In [ ]:
# plot first few images
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_test[i])
    plt.title(y_test[i])

plt.show()

### Build 3 Models
(1) MLP
<br> (2) CNN-1
<br> (3) CNN-2
<br> and compare the three on performance and classification

In [ ]:
#Build 3 models, batchsize=32, Adam, categorical loss, accuracy, 5 epochs

def compile_model(model):
    return model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

def fit_model(model,x_train,y_train,x_test,y_test):
    return model.fit(
        x_train,
        y_train,
        epochs=10,
        batch_size=32,
        validation_data=(x_test,y_test),
        verbose=1
    )

In [ ]:
# (1) Build an MLP: Dense(512,sigmoid), Dense(10,activation)
mlp = Sequential()
mlp.add(Flatten(input_shape=(32,32,3)))
mlp.add(Dense(512,activation='sigmoid',input_shape=(3072,)))
mlp.add(Dense(10,activation='softmax'))
print(mlp.summary())

compile_model(mlp)
h = fit_model(mlp,x_train,y_train,x_test,y_test)
plt.plot(h.history['accuracy'])

plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'], 'r')
plt.legend(['train acc', 'val acc'])
print('train acc: %.2f %% \nval acc: %.2f %%'%(h.history['accuracy'][-1]*100,h.history['val_accuracy'][-1]*100))

plt.plot(np.log10(h.history['loss']))
plt.plot(np.log10(h.history['val_loss']), 'r')
plt.legend(['train loss', 'val loss'])

In [ ]:
# (2) CNN
cnn_2 = Sequential()
cnn_2.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(32,32,3)))
cnn_2.add(Flatten())
cnn_2.add(Dense(512,activation='sigmoid'))
cnn_2.add(Dense(10,activation='softmax'))
print(cnn_2.summary())

compile_model(cnn_2)
h = fit_model(cnn_2,x_train,y_train,x_test,y_test)
plt.plot(h.history['accuracy'])
print('Test accuracy: %.2f %%'%(100*cnn_2.evaluate(x_test,y_test,verbose=0)[1]))

plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'], 'r')
plt.legend(['train acc', 'val acc'])
print('train acc: %.2f %% \nval acc: %.2f %%'%(h.history['accuracy'][-1]*100,h.history['val_accuracy'][-1]*100))

plt.plot(np.log10(h.history['loss']))
plt.plot(np.log10(h.history['val_loss']), 'r')
plt.legend(['train loss', 'val loss'])

In [ ]:
# (3) CNN
cnn_3 = Sequential()
cnn_3.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(32,32,3)))
cnn_3.add(MaxPool2D())
cnn_3.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
cnn_3.add(MaxPool2D())
cnn_3.add(Flatten())
cnn_3.add(Dense(512,activation='sigmoid'))
cnn_3.add(Dropout(0.2))
cnn_3.add(Dense(512,activation='sigmoid'))
cnn_3.add(Dropout(0.2))
cnn_3.add(Dense(10,activation='softmax'))
print(cnn_3.summary())

compile_model(cnn_3)
h = fit_model(cnn_3,x_train,y_train,x_test,y_test)

plt.plot(h.history['accuracy'])
print('Test accuracy: %.2f %%'%(100*cnn_3.evaluate(x_test,y_test, verbose=0)[1]))

plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'], 'r')
plt.legend(['train acc', 'val acc'])
print('train acc: %.2f %% \nval acc: %.2f %%'%(h.history['accuracy'][-1]*100,h.history['val_accuracy'][-1]*100))

plt.plot(np.log10(h.history['loss']))
plt.plot(np.log10(h.history['val_loss']), 'r')
plt.legend(['train loss', 'val loss'])
